# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2023spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/15lGUmT8NpLGtoxRllRWCJyQEjhR1Idcei63YHsDckPE/edit#slide=id.g21fff4e9af6_0_13)　Kaggle: [Link](https://www.kaggle.com/competitions/ml2023spring-hw7/host/sandbox-submissions)　Data: [Link](https://drive.google.com/file/d/1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7/view?usp=sharing)




# Prerequisites

## Download Dataset

In [ ]:
# download link 1
# !gdown --id '1TjoBdNlGBhP_J9C66MOY7ILIrydm7ZCS' --output hw7_data.zip

# download link 2 (if above link failed)
# !gdown --id '1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7' --output hw7_data.zip

# download link 3 (if above link failed)
# !gdown --id '1k2BfGrvhk8QRnr9Xvb04oPIKDr1uWFpa' --output hw7_data.zip

# !unzip -o hw7_data.zip

# !kaggle competitions download -c ml2023spring-hw7

!unzip ml2023spring-hw7.zip   

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

## Install packages

Documentation for the toolkit:
*   https://huggingface.co/transformers/
*   https://huggingface.co/docs/accelerate/index



In [ ]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.26.1
!pip install accelerate==0.16.0

# Kaggle (Fine-tuning)

## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Import Packages

In [1]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(2)

2024-09-20 15:38:01.696026: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-20 15:38:01.703454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 15:38:01.712111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 15:38:01.714761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 15:38:01.721371: I tensorflow/core/platform/cpu_feature_guar

## Load Model and Tokenizer






In [2]:
from transformers import (
  AutoTokenizer,
  AutoModelForQuestionAnswering,
)

# model = AutoModelForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# ---- Strong -----
model = AutoModelForQuestionAnswering.from_pretrained("NchuNLP/Chinese-Question-Answering").to(device)
tokenizer = AutoTokenizer.from_pretrained("NchuNLP/Chinese-Question-Answering")
# ---- Strong -----

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

/home/hoperj/miniconda3/envs/DL/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/hoperj/miniconda3/envs/DL/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-chinese and are new

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

## Read Data

- Training set: 26918 QA pairs
- Dev set: 2863  QA pairs
- Test set: 3524  QA pairs

- {train/dev/test}_questions:
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs:
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions

In [3]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [4]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False)

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors


## Dataset

In [5]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60
        self.max_paragraph_len = 150

        ##### TODO: Change value of doc_stride #####
        self.doc_stride = int(self.max_paragraph_len * 0.25)  # Set stride to 25% of max paragraph length (Medium)

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2

            # ---- Strong -----
            
            # Introduce random offset to prevent learning that answer is always in the middle
            max_offset = self.max_paragraph_len   # We allow up to 1/4 of the max length as offset
            random_offset = np.random.randint(-max_offset, max_offset)  # Random shift between -max_offset and +max_offset

            # Adjust paragraph start based on random offset
            paragraph_start = max(0, min(mid + random_offset - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
                
            # ---- Strong -----

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]

            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)

            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token
        
        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []

            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):

                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]

                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)

                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)

            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len

        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

## Function for Evaluation

In [6]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing
    # Hint: Open your prediction file to see what is wrong

    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]

    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob

        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [7]:
from accelerate import Accelerator

# hyperparameters
num_epoch = 1  # Strong
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
train_batch_size = 8

#### TODO: gradient_accumulation (optional)####
# Note: train_batch_size * gradient_accumulation_steps = effective batch size
# If CUDA out of memory, you can make train_batch_size lower and gradient_accumulation_steps upper
# Doc: https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation
gradient_accumulation_steps = 16

# dataloader
# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

#---- Medium -----
from transformers import get_linear_schedule_with_warmup
from torch.optim.lr_scheduler import LambdaLR

# total training steps
total_steps = len(train_loader) * num_epoch
num_warmup_steps = int(0 * total_steps)  # Set warmup steps to 20% of total steps

# [Hugging Face] Apply linear learning rate decay with warmup
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps
)

# # [PyTorch] Alternative approach: Apply linear learning rate decay without warmup
# # lr_lambda defines how learning rate decays over time (here it's a simple linear decay)
# lr_lambda = lambda step: max(0.0, 1.0 - step / total_steps)
# scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

# ---- Medium -----

# Change "fp16_training" to True to support automatic mixed
# precision training (fp16)
fp16_training = True
if fp16_training:
    accelerator = Accelerator(mixed_precision="fp16")
else:
    accelerator = Accelerator()

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

model.train()


print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for data in tqdm(train_loader):
        # Load all data into GPU
        data = [i.to(device) for i in data]

        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)

        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()

        train_loss += output.loss

        accelerator.backward(output.loss)

        step += 1
        optimizer.step()
        optimizer.zero_grad()

        ##### TODO: Apply linear learning rate decay #####
        scheduler.step()  # Apply learning rate scheduler (Medium)
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model"
model.save_pretrained(model_save_dir)

Start Training ...


/home/hoperj/miniconda3/envs/DL/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 4.329, acc = 0.018
Epoch 1 | Step 200 | loss = 3.066, acc = 0.117
Epoch 1 | Step 300 | loss = 2.860, acc = 0.224
Epoch 1 | Step 400 | loss = 2.691, acc = 0.254
Epoch 1 | Step 500 | loss = 2.518, acc = 0.282
Epoch 1 | Step 600 | loss = 2.266, acc = 0.350
Epoch 1 | Step 700 | loss = 2.206, acc = 0.341
Epoch 1 | Step 800 | loss = 2.011, acc = 0.361
Epoch 1 | Step 900 | loss = 2.004, acc = 0.376
Epoch 1 | Step 1000 | loss = 2.078, acc = 0.366
Epoch 1 | Step 1100 | loss = 2.040, acc = 0.338
Epoch 1 | Step 1200 | loss = 1.992, acc = 0.369
Epoch 1 | Step 1300 | loss = 1.954, acc = 0.381
Epoch 1 | Step 1400 | loss = 1.839, acc = 0.394
Epoch 1 | Step 1500 | loss = 1.842, acc = 0.417
Epoch 1 | Step 1600 | loss = 1.898, acc = 0.431
Epoch 1 | Step 1700 | loss = 1.966, acc = 0.390
Epoch 1 | Step 1800 | loss = 1.720, acc = 0.412
Epoch 1 | Step 1900 | loss = 1.806, acc = 0.429
Epoch 1 | Step 2000 | loss = 1.777, acc = 0.412
Epoch 1 | Step 2100 | loss = 1.856, acc = 0.406
E

  0%|          | 0/2863 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.727


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 2 | Step 100 | loss = 1.620, acc = 0.454
Epoch 2 | Step 200 | loss = 1.613, acc = 0.460
Epoch 2 | Step 300 | loss = 1.603, acc = 0.461
Epoch 2 | Step 400 | loss = 1.697, acc = 0.464
Epoch 2 | Step 500 | loss = 1.505, acc = 0.469
Epoch 2 | Step 600 | loss = 1.630, acc = 0.459
Epoch 2 | Step 700 | loss = 1.527, acc = 0.460
Epoch 2 | Step 800 | loss = 1.591, acc = 0.457
Epoch 2 | Step 900 | loss = 1.594, acc = 0.464
Epoch 2 | Step 1000 | loss = 1.535, acc = 0.469
Epoch 2 | Step 1100 | loss = 1.588, acc = 0.487
Epoch 2 | Step 1200 | loss = 1.510, acc = 0.462
Epoch 2 | Step 1300 | loss = 1.640, acc = 0.442
Epoch 2 | Step 1400 | loss = 1.466, acc = 0.481
Epoch 2 | Step 1500 | loss = 1.539, acc = 0.471
Epoch 2 | Step 1600 | loss = 1.470, acc = 0.454
Epoch 2 | Step 1700 | loss = 1.564, acc = 0.468
Epoch 2 | Step 1800 | loss = 1.572, acc = 0.487
Epoch 2 | Step 1900 | loss = 1.566, acc = 0.480
Epoch 2 | Step 2000 | loss = 1.676, acc = 0.457
Epoch 2 | Step 2100 | loss = 1.516, acc = 0.476
E

  0%|          | 0/2863 [00:00<?, ?it/s]

Validation | Epoch 2 | acc = 0.762
Saving Model ...


## Testing

In [8]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Completed! Result is in result.csv


# GradeScope - Question 2 (In-context learning)

### In-context learning
The example prompt is :
```
請從最後一篇的文章中找出最後一個問題的答案：
文章：<文章1 內容>
問題：<問題1 敘述>
答案：<答案1>
...
文章：<文章n 內容>
問題：<問題n 敘述>
答案：
```

In [11]:
import torch
import random
import numpy as np

# To avoid CUDA_OUT_OF_MEMORY
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Fix random seed for reproducibility
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(2)

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# You can try model with different size
# When using Colab or Kaggle, models with more than 2 billions parameters may
# run out of memory
tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-1.7B")

In [13]:
# To clean model output. If you try different prompts, you may have to fix
# this function on your own
def clean_text(text):
    # Note: When you use unilingual model, the colon may become fullwidth
    text = text.split("答案:")[-1]
    text = text.split(" ")[0]
    return text

In [14]:
import random
import json

with open("hw7_in-context-learning-examples.json", "r") as f:
    test = json.load(f)

# K-shot learning
# Give model K examples to make it achieve better accuracy
# Note: (1) When K >= 4, CUDA_OUT_OFF_MEMORY may occur.
#       (2) The maximum input length of XGLM is 2048
K = 2

question_ids = [qa["id"] for qa in test["questions"]]

with open("in-context-learning-result.txt", "w") as f:
    print("ID,Ground-Truth,Prediction", file = f)
    with torch.no_grad():
        for idx, qa in enumerate(test["questions"]):
            # You can try different prompts
            prompt = "請從最後一篇的文章中找出最後一個問題的答案\n"
            exist_question_indexs = [question_ids.index(qa["id"])]

            # K-shot learning: give the model K examples with answers
            for i in range(K):
                question_index = question_ids.index(qa["id"])
                while(question_index in exist_question_indexs):
                    question_index = random.randint(0, len(question_ids) - 1)
                exist_question_indexs.append(question_index)
                paragraph_id = test["questions"][question_index]["paragraph_id"]
                prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
                prompt += f'問題：{test["questions"][question_index]["question_text"]}\n'
                prompt += f'答案：{test["questions"][question_index]["answer_text"]}\n'

            # The final one question without answer
            paragraph_id = qa["paragraph_id"]
            prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
            prompt += f'問題：{qa["question_text"]}\n'
            prompt += f'答案：'

            inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")
            sample = model.generate(**inputs, max_new_tokens = 20)
            text = tokenizer.decode(sample[0], skip_special_tokens=True)

            # Note: You can delete this line to see what will happen
            text = clean_text(text)

            print(prompt)
            print(f'正確答案: {qa["answer_text"]}')
            print(f'模型輸出: {text}')
            print()

            print(f"{idx},{qa['answer_text']},{text}", file = f)

請從最後一篇的文章中找出最後一個問題的答案
文章：廣州是京廣鐵路，廣深鐵路，廣茂鐵路和廣梅鐵路的終點站。2009年底，武廣客運專線投入運營，多機組列車長980公里，最高時速350公里。2011年1月7日，廣珠城際鐵路投入運營，平均時速200公里。廣州鐵路，長途巴士和渡輪直達香港。廣九快速列車從廣州火車東站出發，直達香港紅磡火車站。總長約182公里，行程大約需要兩個小時。每年都有繁忙的教練從香港的不同乘客點接載乘客。在市中心的珠江北岸有一條渡輪線路，河流居民可以直接過河而無需乘坐公共汽車或步行穿過大橋。每天都有往返南沙碼頭和蓮花山碼頭的高速雙體船。渡輪也開往香港中國客運碼頭和港澳客運碼頭。
問題：廣珠城際鐵路平均每小時可以走多遠？
答案：200公里
文章：自古以來，廣州一直是華南地區的著名商人，擁有2000多年的開放貿易歷史。20世紀70年代末中國大陸改革開放後，廣州經濟發展迅速。2010年，全市地區生產總值10604.48億元，同比增長13％。它成為僅次於上海和北京的第三個進入“萬億元俱樂部”國內生產總值的城市。這也是第一個超過一萬億的經濟總量。首都。根據國務院2005年發布的報告，廣州成為中國第一個進入“發達”狀態的城市。2012年9月，廣州南沙新區獲批，成為第六個國家級開放開發新區。2015年，廣州GDP達到1810.41億元，人均GDP達到138,377.05元。國內生產總值是中國的第三位，人均國內生產總值與西班牙相當。購買力平價水平與發達國家相當。
問題：進入國內生產總值「萬億元俱樂部」的城市第三個為？
答案：廣州
文章：2010年引入的廣州快速交通運輸系統是世界第二大快速運輸系統。每日載客量可達100萬人次。每小時的客流量峰值高達26,900名乘客，僅次於波哥大的快速交通系統。每10秒有一輛公共汽車，每輛公共汽車在一個方向上行駛350小時。該平台包括橋樑，是世界上最長的國家公共汽車快速運輸系統平台，長度為260米。目前，廣州市的出租車和公交車主要以液化石油氣為燃料，部分公交車採用油電，氣電混合技術。2012年底，一輛LNG燃料公共汽車開始啟動。2014年6月，引入了LNG插電式混合動力公交車取代LPG公交車。2007年1月16日，廣州市政府完全禁止在城市地區駕駛摩托車。違反禁令的機動車將被沒收。廣州市交通局聲稱，禁令的實施導致交通擁堵和車禍大大減少。廣州白